# ランダムでレース結果を予測してみる
predict\LGBMRanker01\LGBMRanker01.ipynbで学習させたモデルを使って適当に予測

In [2]:
import pandas as pd
import tqdm

import os
import sys
sys.path.append('../../')
import common.com as common
from common import const,utils
import copy
from datetime import datetime
import scraping.scraping as scraping

In [4]:
# 開催日一覧を取得
from_date = "2024-04"
to_date = "2024-05"
kaisai_date_list = scraping.scrape_kaisai_date(from_=from_date, to_=to_date)

# 開催日に開かれたレースのレースID一覧を取得
race_id_list = scraping.scrape_race_id_list(kaisai_date_list)

import pickle
race_id_list_file_path=f"{const.RACE_ID_LIST_DIR}{from_date}_{to_date}.pickle"
with open(race_id_list_file_path, "wb") as f:
    pickle.dump(race_id_list, f)


# レースID一覧を読み込み
with open(race_id_list_file_path, 'rb') as file:
    race_id_list = pickle.load(file)


# レースID一覧からレース結果を取得
race_datas=[]
for race_id in tqdm.tqdm(race_id_list):
    try:
        race_data = scraping.scrape_race_result(race_id)
        race_datas.extend(race_data)
    except:
        print(f"ERROR: in race_id:{race_id}")
        continue

# レースデータをdataFrame化
race_datas_df = pd.DataFrame(race_datas)
race_datas_df.columns = const.ORIGIN_RACE_DATA_HEADERS

# レースデータをcsvに保存
df = pd.DataFrame(race_datas)

df.columns = const.ORIGIN_RACE_DATA_HEADERS
csv_file_path=race_id_list_file_path.replace('.pickle', '.csv').replace('race_id_lists', 'race_datas')
df.to_csv(csv_file_path, index=False)

In [ ]:
# レースID一覧を読み込み
with open(race_id_list_file_path, 'rb') as file:
    race_id_list = pickle.load(file)

# レースID一覧からレースの払い戻しデータを取得
race_refund_datas=pd.DataFrame()
for race_id in tqdm.tqdm(race_id_list):
    print(race_id)
    data = scraping.scrape_race_refund(race_id)
    race_refund_datas = pd.concat([race_refund_datas,data])

# レースデータをcsvに保存
df = pd.DataFrame(race_refund_datas)

In [3]:
file_path = 'D://work//Programming//data//race_datas//2024-04_2024-05.csv'
race_datas_df = pd.read_csv(file_path)
race_datas_df

,レースID,着順,枠番,馬番,馬の名前,性,齢,斤量,馬のID,騎手の名前,...,日付,開催回詳細,クラス,芝かダートか,距離,回り,馬場状態,天気,競馬場ID,競馬場名
0,202406030501,1,5,7,ショーヴァロンス,牝,3,52.0,2021103908,長浜鴻緒,...,2024年4月6日,3回中山5日目,3歳未勝利,ダ,1800,右,稍,曇,6,中山
1,202406030501,2,7,12,ファイアンス,牝,3,55.0,2021106454,菅原明良,...,2024年4月6日,3回中山5日目,3歳未勝利,ダ,1800,右,稍,曇,6,中山
2,202406030501,3,4,6,サンキャメロン,牝,3,55.0,2021107262,横山武史,...,2024年4月6日,3回中山5日目,3歳未勝利,ダ,1800,右,稍,曇,6,中山
3,202406030501,4,2,2,セイウンウィズミー,牝,3,55.0,2021102941,勝浦正樹,...,2024年4月6日,3回中山5日目,3歳未勝利,ダ,1800,右,稍,曇,6,中山
4,202406030501,5,1,1,ブレイクショット,牝,3,55.0,2021103011,石川裕紀,...,2024年4月6日,3回中山5日目,3歳未勝利,ダ,1800,右,稍,曇,6,中山
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7744,202408031212,12,7,13,ミルトクレイモー,牡,4,58.0,2020100815,小沢大仁,...,2024年5月26日,3回京都12日目,4歳以上3勝クラス,芝,1400,右,良,晴,8,京都
7745,202408031212,13,6,12,チュウワノキセキ,セ,7,58.0,2017105574,三浦皇成,...,2024年5月26日,3回京都12日目,4歳以上3勝クラス,芝,1400,右,良,晴,8,京都
7746,202408031212,14,8,16,タイセイシェダル,牡,6,58.0,2018103021,田口貫太,...,2024年5月26日,3回京都12日目,4歳以上3勝クラス,芝,1400,右,良,晴,8,京都
7747,202408031212,15,3,5,ハギノメーテル,牝,5,56.0,2019100653,今村聖奈,...,2024年5月26日,3回京都12日目,4歳以上3勝クラス,芝,1400,右,良,晴,8,京都


In [4]:
converted_data = copy.deepcopy(race_datas_df)
rank_data_set = set(converted_data['着順'])

# 着順に「降、失、中、取消」など文字が含まれ、数値に変換不可である場合に無効な値（99）を設定
for data in tqdm.tqdm(rank_data_set):    
    if utils.isfloat(data) == False:
        converted_data['着順'] = converted_data['着順'].replace(data, const.INVALID_ORDER)
    else:
        converted_data['着順'] = converted_data['着順'].replace(data, int(data))

# 着順が「取消」の場合、人気が空文字になるので無効な値（99）を設定
converted_data['人気'] = converted_data['人気'].replace('', const.INVALID_ORDER)


# 走破時間を秒数に変換
def convert_time(time):
    try:
        time_obj = datetime.strptime(time, "%M:%S.%f").time()
        time_seconds = time_obj.hour * 3600 + time_obj.minute * 60 + time_obj.second + time_obj.microsecond / 1000000
        return time_seconds
    except ValueError:
        return None
    

converted_data['走破時間'] = converted_data['走破時間'].map(lambda data: convert_time(str(data)))

odds_set = set(converted_data['オッズ'])
for data in tqdm.tqdm(odds_set):    
    if utils.isfloat(data) == False:
        converted_data['オッズ'] = converted_data['オッズ'].replace(data, 0)
    else:
        converted_data['オッズ'] = converted_data['オッズ'].replace(data, float(data))

converted_data['日付'] = converted_data['日付'].map(lambda data: datetime.strptime(data, "%Y年%m月%d日"))

encoded_target_data = copy.deepcopy(converted_data)

# ラベルエンコーディング(LabelEncoder)
from sklearn.preprocessing import LabelEncoder

encode_tar_cols = [
'性',
'斤量',
'芝かダートか',
'回り',
'馬場状態',
'天気',]

le = LabelEncoder()
for col in encode_tar_cols:
    encoded_target_data[col] = le.fit_transform(encoded_target_data[col].values)

  0%|          | 0/21 [00:00<?, ?it/s]C:\Users\ionde\AppData\Local\Temp\ipykernel_20600\1050990908.py:9: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  converted_data['着順'] = converted_data['着順'].replace(data, int(data))
 95%|█████████▌| 2193/2305 [00:01<00:00, 1137.21it/s]C:\Users\ionde\AppData\Local\Temp\ipykernel_20600\1050990908.py:32: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  converted_data['オッズ'] = converted_data['オッズ'].replace(data, float(data))
100%|██████████| 2305/2305 [00:01<00:00, 1163.54it/s]


In [5]:
# 着順関連度で重み付け
ORDER_RELATED = {1:30,2:28,3:26,4:24,5:22}
def get_order_related(order):
    try:
        return ORDER_RELATED[order]
    except KeyError:
        return 0
    


# データのヘッダー（特徴量）
VARIABLE = [
'レースID',
'枠番',
'馬番',
'性',
'齢',
'斤量',
'馬のID',
'騎手のID',
'オッズ',
'人気',
'体重',
'体重変化',
'芝かダートか',
'距離',
'回り',
'馬場状態',
'天気',
'競馬場ID',
'着順関連度']

X_test_predict = pd.DataFrame(encoded_target_data, columns=VARIABLE).drop('着順関連度', axis=1)
y_test_predict = encoded_target_data['着順'].map(get_order_related)


# objectになってしまうので、floatに変換
for col in VARIABLE[:-1]:
    if X_test_predict[col].dtype == 'object':
        if col in ['レースID','馬番','枠番','齢','馬のID','騎手のID','距離','競馬場ID']:
            X_test_predict[col] = X_test_predict[col].astype('int64')
        else:
            X_test_predict[col] = X_test_predict[col].astype(float)


In [6]:
import pickle

# モデルの読み込み
file_dir = const.BASE_DIR+'predict\\LGBMRanker01\\LGBMRanker01.pickle'

model = pickle.load(open(file_dir, 'rb'))

# 任意のレースを予測
_y_pred = model.predict(X_test_predict, num_iteration=model.best_iteration_)


# 予測結果や関連度をDataFrameに連結
result = pd.DataFrame({
    'レースID': X_test_predict['レースID'],
    '馬番': X_test_predict['馬番'],
    '馬のID': X_test_predict['馬のID'],
    '予想スコア':  _y_pred,
    '着順関連度': y_test_predict
})

In [7]:
# クエリーごとにNDCGを計算し、その平均値を算出
from sklearn.metrics import ndcg_score
ndcg_score_result = result.groupby('レースID').apply(lambda d: ndcg_score([d['着順関連度']], [d['予想スコア']], k=5)).mean()
print(ndcg_score_result)

0.6761539534033332


C:\Users\ionde\AppData\Local\Temp\ipykernel_20600\3384237788.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ndcg_score_result = result.groupby('レースID').apply(lambda d: ndcg_score([d['着順関連度']], [d['予想スコア']], k=5)).mean()


In [8]:
# csvに出力
result.groupby('レースID').apply(lambda x: x.sort_values(['予想スコア'], ascending=False)).to_csv('result.csv', index=False)

C:\Users\ionde\AppData\Local\Temp\ipykernel_20600\2306499593.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  result.groupby('レースID').apply(lambda x: x.sort_values(['予想スコア'], ascending=False)).to_csv('result.csv', index=False)


In [9]:
for (id,race) in result.groupby('レースID'):
    print(race)
    break

            レースID  馬番        馬のID     予想スコア  着順関連度
334  202403010101   2  2021103480  0.426926     30
335  202403010101   7  2021100005  0.311884     28
336  202403010101  12  2021107356  0.244796     26
337  202403010101  10  2021100052  0.301125     24
338  202403010101   6  2021105451 -0.998089     22
339  202403010101   9  2021106547 -0.146606      0
340  202403010101  14  2021105001  0.021415      0
341  202403010101  11  2021101732  0.314995      0
342  202403010101  13  2021105219 -2.503648      0
343  202403010101  15  2021102527 -3.176261      0
344  202403010101   8  2021104446 -1.843314      0
345  202403010101   1  2021101257 -3.492932      0
346  202403010101   4  2021105294 -3.176261      0
347  202403010101   5  2021106320 -2.602276      0
348  202403010101   3  2021104081 -2.767144      0
